In [58]:
import sqlalchemy
import pandas as pd
from sqlalchemy.orm import sessionmaker
import requests
import json
from datetime import datetime
import datetime 
import sqlite3

DATABASE_LOCATION = "sqlite://my_played_tracks.sqlite"
USER_ID = "lowriem"

####  PRINT OF A NEW TOKEN EVERY FEW MINTUES AS THEY EXPIRE AT THE FOLLOWING URL: https://developer.spotify.com/console/get-recently-played/?limit=10&after=1596299315000&before=
TOKEN = "BQAqx23MY-vYYZMo92qHzdtRqGAAE4uYkub7jVJ0IHZ1dgv4Zj7UkR43ldVEbhCk_hrNbqGJCNEoKiWcZ2NDDNtW8C6T8iHts_Nj5WKZ6a0MaWMof_0nTCMtuptbxawHm_qYmunlTia58e5eL0xPtIk1cA7PXpHrtL0-BNxT_Yu09QDUCw6Sg05k3stPBx5aEMj0pyYxWw"



def check_if_valid_data(df: pd.DataFrame) -> bool:

  if df.empty:
    print("No songs downloaded, finishing process")
    return False


  if pd.Series(df['played_at']).is_unique:
    pass
  else:
    raise Exception("Primary Key check was violated, duplicates detected")


  if df.isnull().values.any():
    raise Exception("Null value found. Terminated")


  yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
  yesterday = yesterday.replace(hour = 0, minute = 0, second = 0, microsecond = 0)

  timestamps = df["timestamp"].tolist()
  for timestamp in timestamps:
    if datetime.datetime.strptime(timestamp, "%Y-%m-%d") != yesterday:
      raise Exception("One of the values was not in the time constraint")

  return True


if __name__ == "__main__":
  headers = {
      "Accept" : "application/json",
      "Content-Type" : "application/json",
      "Authorization" : "Bearer {token}".format(token=TOKEN)
  }


  today = datetime.datetime.now()
  yesterday = today - datetime.timedelta(days=1)
  yesterday_unix_timestamp = int(yesterday.timestamp()) * 1000

  r = requests.get("https://api.spotify.com/v1/me/player/recently-played?after={time}".format(time = yesterday_unix_timestamp), headers = headers)

  data = r.json()

  song_names =[]
  artist_names = []
  played_at_list = []
  timestamps = []

  for song in data["items"]:
        song_names.append(song["track"]["name"])
        artist_names.append(song["track"]["album"]["artists"][0]["name"])
        played_at_list.append(song["played_at"])
        timestamps.append(song["played_at"][0:10])
  
  song_dict = {
      "song_name" : song_names,
      "artist_names" : artist_names,
      "played_at" : played_at_list,
      "timestamp" : timestamps
  }
  song_df = pd.DataFrame(song_dict, columns = ["song_name", "artist_names", "played_at", "timestamp"])



from tabulate import tabulate




print(tabulate(song_df, headers='keys', tablefmt='psql'))


#validate

if check_if_valid_data(song_df):
  print("Data Valid, proceed to loading stage")


#load
engine = sqlalchemy.create_engine(DATABASE_LOCATION)
conn = sqlite3.connect("my_played_traks.sqlite")
cursor = conn.cursor()

sql_query = """
CREATE TABLE IF NOT EXISTS my_played_tracks(
  song_name VARCHAR(200),
  artist_name VARCHAR(200),
  played_at VARCHAR(200),
  timestamp VARCHAR(200),
  CONSTRAINT primary_key_constraint PRIMARY KEY (played_at)
)

"""

cursor.execute(sql_query)
print("Opened database successfully")

try:
    song_df.to_sql("my_played_tracks", engine, index=False, if_exists='append')
except:
    print("Data already exists in the database")

conn.close()
print("Close database successfully")


+----+-----------------------------------------+----------------+--------------------------+-------------+
|    | song_name                               | artist_names   | played_at                | timestamp   |
|----+-----------------------------------------+----------------+--------------------------+-------------|
|  0 | Darling                                 | D-Block Europe | 2023-02-23T20:51:03.523Z | 2023-02-23  |
|  1 | Us Against the World                    | Strandz        | 2023-02-23T20:49:33.543Z | 2023-02-23  |
|  2 | NIC (feat. Tiggs Da Author)             | Nines          | 2023-02-23T20:30:32.522Z | 2023-02-23  |
|  3 | Who Needs Love                          | Tory Lanez     | 2023-02-23T20:25:20.727Z | 2023-02-23  |
|  4 | Barbie                                  | D-Block Europe | 2023-02-23T18:05:22.483Z | 2023-02-23  |
|  5 | Make A Toast                            | Lil Skies      | 2023-02-23T17:55:14.506Z | 2023-02-23  |
|  6 | Mucky James                   

ArgumentError: ignored